This function computes the outermost closed curve for each family of elliptic curves. This algorithm can be applied to both

In [ ]:
# Import numpy
import numpy as np

# Import shapely library to create polygons and points
from shapely.geometry import Polygon, Point

# Import math tools
from math import sqrt

In [2]:
def find_outermost_closed_curve(Elliptic, T0):
    '''
    Given a family of elliptic Lagrangian diffusion barriers, this function extracts the outermost elliptic Lagrangian diffusion barriers.
    
    Parameters:
        Elliptic:              list, family of elliptic Lagrangian diffusion barriers
        T0:                    array(N, ), transport density T0 of closed elliptic Lagrangian diffusion barrier
    
    Returns:
        x_outermost_elliptic:  list, x-coordinates of outermost elliptic Lagrangian diffusion barriers. len(x_outermost_elliptic) = total nr. of elliptic Eulerian diffusion barriers
        y_outermost_elliptic:  list, y-coordinates of outermost elliptic Lagrangian diffusion barriers. len(x_outermost_elliptic) = total nr. of elliptic Eulerian diffusion barriers
        T0_outermost_elliptic: list, transport density associated to outermost elliptic Lagrangian diffusion barriers. len(x_outermost_elliptic) = total nr. of elliptic Eulerian diffusion barriers
    '''
    
    Poly = []
    Perimeter = []
    _T0 = []
    
    # iterate over transport density
    for i in range(len(T0)):
    
        # iterate over closed curves with transport density given by T0[i]
        if Elliptic[i][0] is not None:
            for j in range(len(Elliptic[i][0])):
                if Elliptic[i][0][j] is not None:
                    
                    # create polygon object of closed curve
                    Poly_object = Polygon(zip(Elliptic[i][0][j], Elliptic[i][1][j]))
                    
                    # calculater perimeter of closed curve and append to perimeter
                    Perimeter.append(Poly_object.length)
                    
                    # append polygon object to list
                    Poly.append(Poly_object)
                    
                    # append transport density corresponding to closed material curve (=Poly_object)
                    _T0.append(T0[i])
            
    # if no closed null geodesic is found then return None
    if len(Perimeter) == 0:
        return [np.nan], [np.nan], [np.nan]
    
    x_elliptic, y_elliptic = [], []
    
    # sort closed curves from maximum to minimum area 
    Perimeter_sorted, Poly_sorted = zip(*sorted(zip(Perimeter.copy(), Poly), reverse = True))
    
    # sort transport density
    Perimeter_sorted, T0_sorted = zip(*sorted(zip(Perimeter.copy(), _T0), reverse = True))
    
    # store closed curve with largest perimeter among all elliptic families
    # coordinates of closed curve
         
    outermost_elliptic = []
    T0_outermost_elliptic = []
    # store outermost elliptic curve (among all the families of elliptic curves)
    
    # iterate over all closed curves(sorted according to area)
    for idx_, Poly_ in enumerate(Poly_sorted):
            
        if idx_ == 0:
            
            outermost_elliptic.append(Poly_sorted[idx_])
            T0_outermost_elliptic.append(T0_sorted[idx_])
        
        else:
        
            # assume that closed curve is a new vortex
            bool_create_vortex = True
            
            # coordinates of closed curve
            coords = np.array(Poly_.exterior.coords)
            
            # iterate over all outermost closed curve
            for v in outermost_elliptic:
            
                if v.contains(Poly_) or v.contains(Point(np.mean(coords[:, 0]), np.mean(coords[:, 1]))):
                    bool_create_vortex = False
                    break
            
            if bool_create_vortex:
            
                outermost_elliptic.append(Poly_)
                T0_outermost_elliptic.append(T0_sorted[idx_])

    x_outermost_elliptic, y_outermost_elliptic = [], []
    
    # Iterate over all outermost elliptic curves and store the x/y coordinates.
    for elliptic in outermost_elliptic:
        
        coords = np.array(elliptic.exterior.coords)
        
        x = coords[:, 0]
        y = coords[:, 1]
        
        x_outermost_elliptic.append(x)
        y_outermost_elliptic.append(y)
    
    return x_outermost_elliptic, y_outermost_elliptic, T0_outermost_elliptic